title: AB Test - card delivery infocards v2   
author: Fabio Schmidt-Fischbach     
date: 2020-05-28     
region: EU   
summary: In a v1 of this experiment, we send infocards with the card status to our users. The next step was to build on top of v1 to also include push notifications that reminded the user about their card order. The push notification is triggered when the card order should have arrived - trying to remind the user to activate the card. The third infocard that reminded users to activate their card on the expected arrival date did not really make a big difference. % of users with card delivery CS contact surge by 0.1% from 3.6% to 3.7% - I estimate the impact to be 1.3k CS contacts over the course of the entire year. % of users with activated card (within 30 days of order) increases by 0.4pp from 70.6% to 71%. Whether this translates into higher activity is hard to tell and we'd need to wait longer to fully understand that. The speed of card activations is roughly the same across both groups. % of users with a card re-order increases by 0.1pp (9% level) increasing the number of re-orders per week by about 25 (1.3k annually).

tags: memberships, enable, card activation, infocards   

In [1]:
import pandas as pd
import altair as alt
import os
import numpy as np

### Summary 

The third infocard that reminded users to activate their card on the expected arrival date did not really make a big difference. 


- CS contacts: % of users with card delivery CS contact surge by 0.1% from 3.6% to 3.7% - I estimate the impact to be 1.3k CS contacts over the course of the entire year. 
- Card activation: % of users with activated card (within 30 days of order) increases by 0.4pp from 70.6% to 71%. Whether this translates into higher activity is hard to tell and we'd need to wait longer to fully understand that. The speed of card activations is roughly the same across both groups. 
- Card re-orders: % of users with a card re-order increases by 0.1pp (9% level) increasing the number of re-orders per week by about 25 (1.3k annually). 





### Setup 

- Goal: Give more transparency on the status of card orders to our users.   
- First step aka variant A: In a v1 of this experiment, we send infocards with the card status to our users. For more details on the first results, consider this [slide deck](https://docs.google.com/presentation/d/1IiR0A-umDa0WOAUrzWVPPpSjpUvcXtMNkoKrFdbooIs/edit?usp=sharing)
- Second step aka variant B: The next step was to build on top of v1 to also include push notifications that reminded the user about their card order. The push notification is triggered when the card order should have arrived - trying to remind the user to activate the card. 

### Sample size 

We only introduced variant B with some delay resulting in a smaller sample size for variant B. Since variant A already proved better than the control we took the control group out of the race later on. 



In [12]:
query = """

with users as ( 
select user_id, created, updated , segment, experiment  
from cm_segmentation 
where experiment = 'PRODUCTION_TRANSIT_INFOCARD' 
)

select  users.*, 
		sf_case.created_date as created_date, 
		count(distinct sf_case.id) as cs_contacts 
from users 
left join sf_contact on sf_contact.user_id::text = users.user_id::text 
left join sf_case on sf_case.contact_id = sf_contact.id and sf_case.created_date >= users.created 
				 and tag_list like '%05 card delivery%'
group by 1,2,3,4,5,6


"""

In [23]:
df = pd.read_csv("cs_contacts.csv")

df = df.groupby(["segment"])["user_id"].agg("nunique").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("segment:N"), y=alt.Y("user_id:Q", axis=alt.Axis(title="Number of users"))
).properties(width=500, height=500, title="Sample size")

alt.Chart(...)

### CS contacts

The % of users that at some point contacted cs about a card delivery issue within 30 days.


In [35]:
df = pd.read_csv("cs_contacts.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["created_date"]) - pd.to_datetime(df["created"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]

# set cs_contacts to 0 if the contact was too late.
df.loc[df["date_diff"] > 30, "cs_contacts"] = 0

# get cs contacts per user.
df = df.groupby(["segment", "user_id"])["cs_contacts"].agg("max").reset_index()

df = df.groupby(["segment"])["cs_contacts"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("segment:N"),
    y=alt.Y(
        "cs_contacts:Q",
        axis=alt.Axis(
            title="% of users with CS contact on card delivery within 30 days",
            format="%",
        ),
    ),
).properties(
    width=500, height=500, title="% of users that contacted CS at some point (30 days)"
)

alt.Chart(...)

In [ ]:
df.head()

When do these CS contacts happen? 

- Control group had more contacts in the first 14 days. 
- Especially, in variant B we see a spike in CS contacts around the time when the delivery is due. These could be customers where delivery failed - their app tells them the card should be there but it might not have arrived. 

In [34]:
df = pd.read_csv("cs_contacts.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["created_date"]) - pd.to_datetime(df["created"])
).dt.days

# replace missing date_diff with v large value
df.loc[df["date_diff"].isna() == True, "date_diff"] = 100000

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]

# get first cs contact of a user.
df = df.groupby(["segment", "user_id"])["date_diff"].agg("min").reset_index()
df = df.groupby(["segment", "date_diff"])["user_id"].agg("nunique").reset_index()

df["perc"] = 100 * df["user_id"] / df.groupby(["segment"])["user_id"].transform("sum")

alt.Chart(
    df.loc[(df["date_diff"] <= 40) & (df["date_diff"] >= 0), :]
).mark_line().encode(
    x=alt.X("date_diff:N", axis=alt.Axis(title="Days since card order")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of total CS contacts within group")),
    color="segment:N",
).properties(
    width=700, height=500, title="% of total card delivery CS contacts within group"
)

alt.Chart(...)

To be sure, we compare A and B only for the time frame when they co-existed. This should resolve any time effects that are introduced by having variant A running for longer than B. 

- % of users with CS contacts surges by 0.1% in variant B. 
- with 25k card orders per week, this increases # of cs contacts roughly by 25 CS contacts a week - 1300 CS contacts a year. 

In [96]:
df = pd.read_csv("cs_contacts.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["created_date"]) - pd.to_datetime(df["created"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set cs_contacts to 0 if the contact was too late.
df.loc[df["date_diff"] > 30, "cs_contacts"] = 0

# get cs contacts per user.
df = df.groupby(["segment", "user_id"])["cs_contacts"].agg("max").reset_index()

df = df.groupby(["segment"])["cs_contacts"].agg("mean").reset_index()

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"]), :]).mark_bar().encode(
    x=alt.X("segment:N"),
    y=alt.Y(
        "cs_contacts:Q",
        axis=alt.Axis(
            title="% of users with CS contact on card delivery within 30 days",
            format="%",
        ),
    ),
).properties(
    width=500, height=500, title="% of users that contacted CS at some point (30 days)"
)

alt.Chart(...)

In [95]:
df.head()

,segment,cs_contacts
0,CONTROL,0.044734
1,VARIANT_A,0.036542
2,VARIANT_B,0.037332


In [15]:
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv("cs_contacts.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["created_date"]) - pd.to_datetime(df["created"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set cs_contacts to 0 if the contact was too late.
df.loc[df["date_diff"] > 30, "cs_contacts"] = 0

# get cs contacts per user.
df = df.groupby(["segment", "user_id"])["cs_contacts"].agg("max").reset_index()
df = (
    df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"]), :]
    .groupby("segment")["cs_contacts"]
    .agg(["count", "sum"])
    .reset_index()
)

# run z test. (two sided)
stat, pval = proportions_ztest(df["sum"], df["count"])

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not significant.")

The z-score for this test is -0.49 which corresponds to a p-value of 0.6209
The difference is not significant.


In [46]:
df = pd.read_csv("cs_contacts.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["created_date"]) - pd.to_datetime(df["created"])
).dt.days

# replace missing date_diff with v large value
df.loc[df["date_diff"].isna() == True, "date_diff"] = 100000

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# get first cs contact of a user.
df = df.groupby(["segment", "user_id"])["date_diff"].agg("min").reset_index()
df = df.groupby(["segment", "date_diff"])["user_id"].agg("nunique").reset_index()

df["perc"] = 100 * df["user_id"] / df.groupby(["segment"])["user_id"].transform("sum")

alt.Chart(
    df.loc[(df["date_diff"] <= 40) & (df["date_diff"] >= 0), :]
).mark_line().encode(
    x=alt.X("date_diff:N", axis=alt.Axis(title="Days since card order")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of total CS contacts within group")),
    color="segment:N",
).properties(
    width=700, height=500, title="% of total card delivery CS contacts within group"
)

alt.Chart(...)

### Card activation 

The next step is to understand card activation across groups. Again we need to be aware off time trends in the data. Let's visualize the card activation rates by group and across time first. 

In [36]:
query = """ 
with users as ( 
select cm_segmentation.user_id, 
		cm_segmentation.created, 
		cm_segmentation.updated , 
		segment, 
		experiment, 
		user_created, 
		is_mau,
		all_accounts_balance_cents::float/100 as balance 
from cm_segmentation 
inner join dbt.zrh_users as zu on zu.user_id = cm_segmentation.user_id  
where experiment = 'PRODUCTION_TRANSIT_INFOCARD'
)

select users.*, 
		min(card_activated) as card_activation, 
		min(order_date) as card_order,
		count(distinct zc.id) as cards 
from users 
inner join dbt.zrh_cards as zc on zc.user_created = users.user_created and zc.order_date >= users.created 
group by 1,2,3,4,5,6,7,8 




"""

In [77]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

df["created"] = pd.to_datetime(df["created"]).dt.week.astype(str)

# get card activation rate per order.
df = df.groupby(["segment", "created"])["card_activate"].agg("mean").reset_index()

alt.Chart(df).mark_line().encode(
    x=alt.X("created:N"),
    y=alt.Y(
        "card_activate:Q",
        axis=alt.Axis(
            title="% of users with activated card within 30 days", format="%"
        ),
    ),
    color="segment:N",
).properties(
    width=500, height=500, title="% of users with activated card within 30 days"
)

alt.Chart(...)

There seems to be a slight negative trend in activations in recent weeks. To properly compare variant B and variant A in the data, we will want to drop variant A traffic from times when variant B was not live yet. 
- % of users with active card increases by 0.4pp from 70.6% to 71%.

In [7]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

# get card activation rate per order.
df = df.groupby(["segment"])["card_activate"].agg("mean").reset_index()

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_bar().encode(
    x=alt.X("segment:N", axis=alt.Axis(title="Segment")),
    y=alt.Y(
        "card_activate:Q",
        axis=alt.Axis(title="% of users with card activated within 30 days"),
    ),
).properties(
    width=500, height=500, title="% of users with card activated within 30 days"
)

alt.Chart(...)

In [8]:
df.head()

,segment,card_activate
0,CONTROL,0.733333
1,VARIANT_A,0.706230
2,VARIANT_B,0.710497


In [16]:
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

df = (
    df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"]), :]
    .groupby("segment")["card_activate"]
    .agg(["count", "sum"])
    .reset_index()
)

# run z test. (two sided)
stat, pval = proportions_ztest(df["sum"], df["count"])

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not significant.")

The z-score for this test is -0.32 which corresponds to a p-value of 0.7498
The difference is not significant.


Do users activate their cards at different times? 

In [80]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

# get card activation rate per order.
df = df.groupby(["segment", "date_diff"])["card_activate"].agg("count").reset_index()
df["perc"] = (
    100
    * df["card_activate"]
    / df.groupby(["segment"])["card_activate"].transform("sum")
)


alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_line().encode(
    x=alt.X("date_diff:N", axis=alt.Axis(title="Days since order")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of card activations")),
    color="segment:N",
).properties(width=500, height=500, title="Time to activate a card")

alt.Chart(...)

Avg. activation time increases slightly from 6.6 to 6.8 days on average. 

In [92]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

# get card activation rate per order.
df = df.groupby(["segment"])["date_diff"].agg("mean").reset_index()

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_bar().encode(
    x=alt.X("segment:N", axis=alt.Axis(title="Days since order")),
    y=alt.Y("date_diff:Q", axis=alt.Axis(title="Avg. days to activate")),
).properties(width=500, height=500, title="Avg. days to activate card")

alt.Chart(...)

In [93]:
df.head()

,segment,date_diff
0,CONTROL,6.874296
1,VARIANT_A,6.675528
2,VARIANT_B,6.824013


The next step is to consider re-orders. We simply count the number of orders per user in the experiment and compute the re-order rate by avg. card orders per user - 1. 
- Re-orders rise by 0.2pp. 


In [19]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

df = df.groupby(["segment"])["cards"].agg("mean").reset_index()

df["cards"] = df["cards"] - 1

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_bar().encode(
    x=alt.X("segment:N", axis=alt.Axis(title="Days since order")),
    y=alt.Y("cards:Q", axis=alt.Axis(title="% of users with re-order", format="%")),
).properties(width=500, height=500, title="Re-order rate per user")

alt.Chart(...)

In [20]:
df.head()

,segment,cards
0,CONTROL,0.102778
1,VARIANT_A,0.088567
2,VARIANT_B,0.089628


In [17]:
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

df["re_order"] = 0
df.loc[df["cards"] > 1, "re_order"] = 1

df = (
    df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"]), :]
    .groupby("segment")["re_order"]
    .agg(["count", "sum"])
    .reset_index()
)

# run z test. (two sided)
stat, pval = proportions_ztest(df["sum"], df["count"])

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not significant.")

The z-score for this test is 0.32 which corresponds to a p-value of 0.7487
The difference is not significant.


When are these re-orders occuring? 

In [ ]:
query = """ 

with users as ( 
select cm_segmentation.user_id, 
		cm_segmentation.created, 
		cm_segmentation.updated , 
		segment, 
		experiment, 
		user_created, 
		is_mau,
		all_accounts_balance_cents::float/100 as balance 
from cm_segmentation 
inner join dbt.zrh_users as zu on zu.user_id = cm_segmentation.user_id  
where experiment = 'PRODUCTION_TRANSIT_INFOCARD'
)

select users.*, 
		card_activated,
		order_date,
		row_number() over(partition by users.user_created order by order_date) as order_no  
from users 
inner join dbt.zrh_cards as zc on zc.user_created = users.user_created and zc.order_date >= users.created 




"""

In [40]:
df = pd.read_csv("card_orders.csv")

df = df.loc[df["order_no"] <= 2, :]

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["order_date"]) - pd.to_datetime(df["created"])
).dt.days
df.loc[df["order_no"] == 1, "date_diff"] = 51

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

df = df.groupby(["segment", "date_diff"])["user_id"].agg("count").reset_index()

df["perc"] = 100 * df["user_id"] / df.groupby(["segment"])["user_id"].transform("sum")

alt.Chart(df.loc[df["date_diff"] < 20, :]).mark_bar().encode(
    x=alt.X("date_diff:N", axis=alt.Axis(title="Days since first order")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of users with re-order")),
    column="segment:N",
).properties(width=200, height=500, title="Timing of re-orders")

alt.Chart(...)

#### Activity 


In [41]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

# get card activation rate per order.
df = df.groupby(["segment"])["is_mau"].agg("mean").reset_index()

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_bar().encode(
    x=alt.X("segment:N"),
    y=alt.Y("is_mau:Q", axis=alt.Axis(title="% of users that are MAUs", format="%")),
).properties(width=500, height=500, title="% of users that are MAUs")

alt.Chart(...)

In [42]:
df.head()

,segment,is_mau
0,CONTROL,0.827778
1,VARIANT_A,0.819303
2,VARIANT_B,0.803269


In [43]:
df = pd.read_csv("card_activation.csv")

# count days since experiment start.
df["date_diff"] = (
    pd.to_datetime(df["card_activation"]) - pd.to_datetime(df["card_order"])
).dt.days

# drop users that joined in the last 30 days -- too young.
df["age"] = (pd.to_datetime("today") - pd.to_datetime(df["created"])).dt.days
df = df.loc[df["age"] >= 30, :]
df = df.loc[pd.to_datetime(df["created"]) >= pd.to_datetime("2020-05-18"), :]

# set card_activate to 1 if the user activated card within 30 days
df["card_activate"] = 0
df.loc[df["date_diff"] <= 30, "card_activate"] = 1

# get card activation rate per order.
df = df.groupby(["segment"])["balance"].agg("median").reset_index()

alt.Chart(df.loc[df["segment"].isin(["VARIANT_A", "VARIANT_B"])]).mark_bar().encode(
    x=alt.X("segment:N"),
    y=alt.Y("balance:Q", axis=alt.Axis(title="Median balance of users")),
).properties(width=500, height=500, title="Median balance of users")

alt.Chart(...)

In [44]:
df.head()

,segment,balance
0,CONTROL,126.00
1,VARIANT_A,117.52
2,VARIANT_B,128.55


#### US results 

- Sample size: https://metabase-main.tech26.us/question/835 / card orders over time : https://metabase-main.tech26.us/question/836
- CS contacts: did not work. 
- Card activation over time https://metabase-main.tech26.us/question/837 
- Card activation + re-order: https://metabase-main.tech26.us/question/838
